In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind_from_stats
from scipy.stats import t, mannwhitneyu, chi2_contingency

mock_path = '/openface'

column_list = ['AU01', 'AU02', 'AU04', 'AU05', 'AU06', 'AU07', 'AU09', 'AU12', 'AU14', 'AU15', 'AU17', 'AU20', 'AU23', 'AU25', 'AU26']
index_list = list(range(1, 301))

mock_output_path = 'F:/FacialExpression/MicroExpression'
mock_analysis_output_path = 'F:/FacialExpression/'

T_total_result = np.zeros((144, 300, 15))
F_total_result = np.zeros((72, 300, 15))
f_idx = 0
t_idx = 0

for f in sorted(os.listdir(mock_path)):
    df = pd.read_csv(mock_path + '/' + f)
    Dataframe = df.iloc[:, 696:713]
    ME_AUs = [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15]
    Dataframe = Dataframe.iloc[:, ME_AUs]
    Dataframe = Dataframe.to_numpy()
    result = np.zeros((300, 15))

    for AU in range(15):
        for frame_len in range(2, 300):
            ME_count = 0
            Exp_count = 0
            for i in range(0, frame_len - 2):
                if float(Dataframe[i, AU]) * float(Dataframe[i + 1, AU]) == 1 and Exp_count < 15:
                    Exp_count += 1
                elif float(Dataframe[i, AU]) * float(Dataframe[i + 1, AU]) == 1 and Exp_count >= 15:
                    Exp_count = 0
                elif Exp_count > 1 and Exp_count < 15 and float(Dataframe[i, AU]) * float(Dataframe[i + 1, AU]) == 0:
                    ME_count += 1
                    Exp_count = 0
            result[frame_len, AU] = ME_count
    ME_count_df = pd.DataFrame(result, columns=column_list, index=index_list)

    if not os.path.exists(mock_output_path):
        os.makedirs(mock_output_path)
    ME_count_df.to_csv(mock_output_path + '/' + f.split('.')[0] + '.csv')

    if f.split('.')[0].split('_')[-1] == 'T':
        T_total_result[t_idx, :, :] = result
        t_idx += 1
    elif f.split('.')[0].split('_')[-1] == 'F':
        F_total_result[f_idx, :, :] = result
        f_idx += 1

column_list = ['AU01', 'AU02', 'AU04', 'AU05', 'AU06', 'AU07', 'AU09', 'AU12', 'AU14', 'AU15', 'AU17', 'AU20', 'AU23', 'AU25', 'AU26', 'Sum']
index_list = list(range(1, 301))

statistics = {}
confidence_interval = 0.95

T_sum = T_total_result.sum(axis=2)
F_sum = F_total_result.sum(axis=2)
T_total_result = np.concatenate((T_total_result, T_sum[:, :, None]), axis=2)
F_total_result = np.concatenate((F_total_result, F_sum[:, :, None]), axis=2)

ME_stat_result_pvalue = np.zeros((300, 16))
ME_stat_result_tstat = np.zeros((300, 16))

for AU in range(16):
    for frame in range(300):
        observation_A = T_total_result[:, frame, AU]
        observation_B = F_total_result[:, frame, AU]
        observation_statistics = {}

        truth_mean = np.mean(T_total_result[:, frame, AU])
        truth_std = np.std(T_total_result[:, frame, AU])
        truth_sample_size = T_total_result.shape[0]

        deception_mean = np.mean(F_total_result[:, frame, AU])
        deception_std = np.std(F_total_result[:, frame, AU])
        deception_sample_size = F_total_result.shape[0]

        t_statistic, p_value = ttest_ind_from_stats(
            truth_mean, truth_std, truth_sample_size,
            deception_mean, deception_std, deception_sample_size
        )

        ME_stat_result_pvalue[frame, AU] = p_value
        ME_stat_result_tstat[frame, AU] = t_statistic

df1 = pd.DataFrame(ME_stat_result_pvalue, columns=column_list, index=index_list)
df1.to_csv(mock_analysis_output_path + '/' + 'mock_ME_stat_result_pvalue.csv', index=True)
df2 = pd.DataFrame(ME_stat_result_tstat, columns=column_list, index=index_list)
df2.to_csv(mock_analysis_output_path + '/' + 'mock_ME_stat_result_tstat.csv', index=True)
```